# Chains
Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components. Chains allow us to combine multiple components together to create a single, coherent application.

In [ ]:
!pip install python-dotenv chromadb langchain openai

In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["service"],
    template="What is a good name for a company that provides {service}?",
)

We can now create a very simple chain that will take user input, format the prompt with it, and then send it to the LLM.

In [2]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("SaaS solutions"))



1. CloudConnect Solutions
2. CloudCatalyst
3. SaaS-Tech
4. CloudMinds
5. SimpliCloud Services
6. CloudBoost Solutions
7. Skyway Solutions
8. Nimbus Technologies
9. CloudFactory
10. StreamTech Solutions


If there are multiple variables, you can input them all at once using a dictionary.

In [3]:
prompt = PromptTemplate(
    input_variables=["company", "service"],
    template="What is a good name for a {company} that provides {service}?",
)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'company': "AI Startup",
    'service': "cloud services and automation"
    }))



Cloud Autom8.
